In [60]:
import os
import warnings
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt
from collections import UserDict
from sklearn.preprocessing import MinMaxScaler
from IPython.display import Image
import tensorflow as tf
%matplotlib inline

df=pd.read_csv('~/Desktop/googleflu 1.csv')

pd.options.display.float_format = '{:,.2f}'.format
np.set_printoptions(precision=2)
warnings.filterwarnings("ignore")

In [61]:
df['ds']=pd.to_datetime(df['Date'])
df['y']=df['California']
df.head()

,Date,United States,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,...,"Norfolk, VA","Reston, VA","Richmond, VA","Bellevue, WA","Seattle, WA","Spokane, WA","Madison, WI","Milwaukee, WI",ds,y
0,2003-09-28,902,477,nan,606,nan,929,233,223,nan,...,948,nan,1035,nan,668,nan,622,452,2003-09-28,929
1,2003-10-05,952,501,nan,663,nan,849,251,243,nan,...,963,nan,1135,nan,787,nan,626,449,2003-10-05,849
2,2003-10-12,1092,492,nan,700,nan,1032,283,261,nan,...,970,nan,1170,nan,994,nan,661,437,2003-10-12,1032
3,2003-10-19,1209,533,nan,819,nan,1084,310,268,nan,...,1011,nan,1342,nan,999,nan,649,487,2003-10-19,1084
4,2003-10-26,1249,594,nan,959,nan,989,344,334,nan,...,1009,nan,1354,nan,1297,nan,657,499,2003-10-26,989


In [62]:
df_flu = df[['Date', 'California']]
df_flu.head()

,Date,California
0,2003-09-28,929
1,2003-10-05,849
2,2003-10-12,1032
3,2003-10-19,1084
4,2003-10-26,989


In [63]:
df_flu['Date'] = pd.to_datetime(df_flu['Date'])
df_flu.head()

,Date,California
0,2003-09-28,929
1,2003-10-05,849
2,2003-10-12,1032
3,2003-10-19,1084
4,2003-10-26,989


In [64]:
df_flu= df_flu.set_index(pd.DatetimeIndex(df_flu['Date']))

In [65]:
valid_start_dt = '2013-09-01 00:00:00'
test_start_dt = '2014-09-01 00:00:00'

In [66]:
T = 6
HORIZON = 1

In [67]:
train = df_flu.loc[df_flu.index<valid_start_dt, 'California'].to_frame('California')
train.head()

,California
Date,
2003-09-28,929
2003-10-05,849
2003-10-12,1032
2003-10-19,1084
2003-10-26,989


In [68]:
scaler = MinMaxScaler()
train['California'] = scaler.fit_transform(train)
train.head(10)

,California
Date,
2003-09-28,0.02
2003-10-05,0.02
2003-10-12,0.03
2003-10-19,0.03
2003-10-26,0.03
2003-11-02,0.05
2003-11-09,0.08
2003-11-16,0.10
2003-11-23,0.16


In [69]:
train_shifted = train.copy()
train_shifted['y_t+1'] = train_shifted['California'].shift(-1, freq='W')
for t in range(1, T+1):
    train_shifted['Kansas'+str(T-t)] = train_shifted['California'].shift(T-t, freq='W')
y_col = 'y_t+1'
X_cols = ['flu_t-5',
             'flu_t-4',
             'flu_t-3',
             'flu_t-2',
             'flu_t-1',
             'flu_t']
train_shifted.columns = ['flu_original']+[y_col]+X_cols

In [70]:
train_shifted = train_shifted.dropna(how='any')
train_shifted.head(5)

,flu_original,y_t+1,flu_t-5,flu_t-4,flu_t-3,flu_t-2,flu_t-1,flu_t
Date,,,,,,,,
2003-11-02,0.05,0.08,0.02,0.02,0.03,0.03,0.03,0.05
2003-11-09,0.08,0.10,0.02,0.03,0.03,0.03,0.05,0.08
2003-11-16,0.10,0.16,0.03,0.03,0.03,0.05,0.08,0.10
2003-11-23,0.16,0.31,0.03,0.03,0.05,0.08,0.10,0.16
2003-11-30,0.31,0.58,0.03,0.05,0.08,0.10,0.16,0.31


In [71]:
y_train = train_shifted[y_col].as_matrix()
X_train = train_shifted[X_cols].as_matrix()

In [72]:
X_train = X_train.reshape(X_train.shape[0], T, 1)

In [73]:
look_back_dt = dt.datetime.strptime(valid_start_dt, '%Y-%m-%d %H:%M:%S') - dt.timedelta(hours=T-1)
valid = df_flu.loc[df_flu.index<test_start_dt, 'California'].to_frame('California')
valid.head()

,California
Date,
2003-09-28,929
2003-10-05,849
2003-10-12,1032
2003-10-19,1084
2003-10-26,989


In [74]:
valid['California'] = scaler.transform(valid)
valid.head()

,California
Date,
2003-09-28,0.02
2003-10-05,0.02
2003-10-12,0.03
2003-10-19,0.03
2003-10-26,0.03


In [75]:
valid_shifted = valid.copy()
valid_shifted['y+1'] = valid_shifted['California'].shift(-1, freq='W')
for t in range(1, T+1):
    valid_shifted['California'+str(T-t)] = valid_shifted['California'].shift(T-t, freq='W')
valid_shifted = valid_shifted.dropna(how='any')
valid_shifted.head(3)

,California,y+1,California5,California4,California3,California2,California1,California0
Date,,,,,,,,
2003-11-02,0.05,0.08,0.02,0.02,0.03,0.03,0.03,0.05
2003-11-09,0.08,0.10,0.02,0.03,0.03,0.03,0.05,0.08
2003-11-16,0.10,0.16,0.03,0.03,0.03,0.05,0.08,0.10


In [76]:
y_valid = valid_shifted['y+1'].as_matrix()
X_valid = valid_shifted[['California'+str(T-t) for t in range(1, T+1)]].as_matrix()
X_valid = X_valid.reshape(X_valid.shape[0], T, 1)

In [92]:
from tf.keras.models import Model, Sequential
from tf.keras.layers import GRU, Dense
from tf.keras.callbacks import EarlyStopping

ModuleNotFoundError: No module named 'tf'

In [78]:
LATENT_DIM = 5
BATCH_SIZE = 32
EPOCHS = 50

In [79]:
model = Sequential()
model.add(GRU(LATENT_DIM, input_shape=(T, 1)))
model.add(Dense(HORIZON))

NameError: name 'Sequential' is not defined

In [80]:
model.compile(optimizer='RMSprop', loss='mse')

NameError: name 'model' is not defined

In [81]:
model.summary()

NameError: name 'model' is not defined

In [82]:
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=2)

NameError: name 'EarlyStopping' is not defined

In [83]:
history = model.fit(X_train,
                    y_train,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    validation_data=(X_valid, y_valid),
                    callbacks=[earlystop],
                    verbose=1)

NameError: name 'model' is not defined

In [84]:
plot_df = pd.DataFrame.from_dict({'train_loss':history.history['loss'], 'val_loss':history.history['val_loss']})
plot_df.plot(logy=True, figsize=(10,10), fontsize=12)
plt.xlabel('epoch', fontsize=12)
plt.ylabel('loss', fontsize=12)
plt.show()

NameError: name 'history' is not defined

In [85]:
look_back_dt = dt.datetime.strptime(test_start_dt, '%Y-%m-%d %H:%M:%S') - dt.timedelta(hours=T-1)
test = df_flu.copy()[test_start_dt:][['California']]
test['California'] = scaler.transform(test)
test_shifted = test.copy()
test_shifted['y_t+1'] = test_shifted['California'].shift(-1, freq='W')
for t in range(1, T+1):
    test_shifted['California'+str(T-t)] = test_shifted['California'].shift(T-t, freq='W')
test_shifted = test_shifted.dropna(how='any')
y_test = test_shifted['y_t+1'].as_matrix()
X_test = test_shifted[['California'+str(T-t) for t in range(1, T+1)]].as_matrix()
X_test = X_test.reshape(X_test.shape[0], T, 1)

In [86]:
predictions = model.predict(X_test)
predictions

NameError: name 'model' is not defined

In [87]:
eval_df = pd.DataFrame(predictions, columns=['t+'+str(t) for t in range(1, HORIZON+1)])
eval_df['timestamp'] = test_shifted.index
eval_df = pd.melt(eval_df, id_vars='timestamp', value_name='prediction', var_name='h')
eval_df['actual'] = np.transpose(y_test).ravel()
eval_df[['prediction', 'actual']] = scaler.inverse_transform(eval_df[['prediction', 'actual']])
eval_df.head()

NameError: name 'predictions' is not defined

In [88]:
def mape(predictions, actuals):
    """Mean absolute percentage error"""
    return ((predictions - actuals).abs() / actuals).mean()

In [89]:
mape(eval_df['prediction'], eval_df['actual'])

NameError: name 'eval_df' is not defined

In [90]:
eval_df[eval_df.timestamp<'2016-11-08'].plot(x='timestamp', y=['prediction', 'actual'], style=['r', 'b'], figsize=(15, 8))
plt.xlabel('timestamp', fontsize=12)
plt.ylabel('load', fontsize=12)
plt.show()

NameError: name 'eval_df' is not defined